In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from collections import Counter
from sklearn.model_selection import KFold,train_test_split
from sklearn.linear_model import ElasticNet,LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
import xgboost as xgb


pd.set_option('max_columns',135)
pd.set_option('max_rows',200)
sns.set(color_codes=True)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ROUTE = '/content/drive/My Drive/Colab Notebooks/Real_Estate_AVM/data/'
# training_data = pd.read_csv(ROUTE+'train_avm.csv').iloc[:,1:] #(37687, 134)
# testing_data =  pd.read_csv(ROUTE+'test_avm.csv').iloc[:,1:] #(6301, 133)

training_data_impute = pd.read_csv(ROUTE+'train_avm_impute.csv').iloc[:,1:] #(37687, 136)
testing_data_impute =  pd.read_csv(ROUTE+'test_avm_impute.csv').iloc[:,1:] #(6301, 135)



# Function

In [ ]:
def preprocessing1(df):
  # tot_price移到首位
    col_array = df.columns
    idx = np.where(col_array=='total_price')
    col_ls = col_array.tolist()
    item = col_ls.pop(idx[0][0])
    col_ls.insert(0,item)

    df = df[col_ls]

  # 移除有缺失值的row
    na_row = np.where(df['village_income_median'].isnull()==True)[0].tolist()
    df = df.drop(na_row,axis=0)

  # 移除缺失值過多的column(14個)
    del_col_df = df.drop(labels=['doc_Rate','master_rate','bachelor_rate','jobschool_rate','highschool_rate','junior_rate','elementary_rate','born_rate','death_date','marriage_rate','divorce_rate','parking_area','parking_price','txn_floor'],axis=1)

  # dummy
    idx_ls = df.iloc[:,0:18].nunique().index
    n = df.iloc[:,0:18].nunique().values
    cat_ls=[]
    for i in range(len(idx_ls)):
      if n[i] <15:
        cat_ls.append(idx_ls[i])
        # print(f'{idx_ls[i]}:{n[i]}')
    df_dummy = pd.get_dummies(data=del_col_df,columns=cat_ls)
    print(df_dummy.shape)
    return df_dummy

In [ ]:
# tot_price移到首位
def preprocessing2(df):
  # 將total_price移至首位
    try:
      col_array = df.columns
      idx = np.where(col_array=='total_price')
      col_ls = col_array.tolist()
      item = col_ls.pop(idx[0][0])
      col_ls.insert(0,item)
      df = df[col_ls]
    except:
      pass
  # 處理缺失值
    df['village_income_median'] = df['vim_impute']
    df['txn_floor'] = df['txnfloor_impute1']
    df = df.drop(['vim_impute','txnfloor_impute1','parking_area','parking_price','doc_Rate','master_rate','bachelor_rate','jobschool_rate','highschool_rate','junior_rate','elementary_rate','born_rate','death_date','marriage_rate','divorce_rate'],axis=1)

  # 類別變數縮減類別
    df['building_type'] = df.building_type.replace({3:0,4:1})
    df['building_material'] = df.building_material.replace({8:1,11:3,10:9})
    df['building_use'] = df.building_use.replace({1:0,2:0,3:0,7:0,8:0,10:5}) #(,121)
  
  # del material_5(只有一個)
    idx_m5 = df[df.building_material==5].index
    df = df.drop(labels=idx_m5)

  # dummy
    col_array = df.columns
    idx_village =  np.where(col_array=='village')[0][0]
    idx_ls = df.iloc[:,0:idx_village].nunique().index
    n = df.iloc[:,0:idx_village].nunique().values
    cat_ls=[]
    for i in range(len(idx_ls)):
      if n[i] <15:
          cat_ls.append(idx_ls[i])
          # print(f'{idx_ls[i]}:{n[i]}')
    df_dummy = pd.get_dummies(data=df,columns=cat_ls)
    print(df_dummy.shape)

    return df_dummy

In [ ]:
training_data_dummy = preprocessing2(training_data_impute)
print('=====')
testing_data_dummy = preprocessing2(testing_data_impute)


(37686, 142)
=====
(6301, 141)


In [ ]:
# training_data_dummy.to_csv(ROUTE+'training_data_dummy2.csv',header=True,index=False)
# testing_data_dummy.to_csv(ROUTE+'testing_data_dummy2.csv',header=True,index=False)